**MultiAgentic RAG for Mental Healthcare**:

The notebook implements a system for answering user queries related to mental health. It utilizes a combination of a Frequently Asked Questions (FAQ) knowledge base and internet search (specifically Wikipedia) to provide relevant information.

**Workflow**:

**Installation**: Installs necessary libraries like langchain, tiktoken, openai, faiss-cpu, wikipedia, bs4, and requests.

**Initialization**: Sets up the OpenAI API key, loads pre-defined FAQs, creates a vector store for semantic search using FAISS, and initializes the language model (LLM) with gpt-4.

**Routing**: A route_query function determines whether the user query should be addressed by the FAQ knowledge base or an internet search based on the query's content.

**FAQ Agent**: The faq_lookup function searches the FAQ knowledge base for the most similar question to the user's query and returns the corresponding answer if a match is found above a certain threshold.

**Search Agent**: If the routing agent directs the query to an internet search, the internet_search_wiki function queries Wikipedia for relevant articles. It extracts snippets from the top articles and uses the LLM to summarize the information, providing citations. If no Wikipedia results are found, it falls back to a simple Google WebView scrape.

**Orchestrator**: The answer_query function orchestrates the entire process by first routing the query, then invoking either the FAQ or search agent based on the route, and finally returning the answer.

**Example Usage**: The notebook includes example queries demonstrating how to use the system to answer questions about anxiety attacks and depression.

In essence, the notebook provides a basic framework for a mental health chatbot that can leverage both pre-defined knowledge and internet search to address user inquiries.

In [ ]:
# ─── 1) Install dependencies ────────────────────────────────────────────────
!pip install --quiet langchain_community
!pip install tiktoken
!pip install --quiet openai langchain faiss-cpu wikipedia bs4 requests

In [ ]:
# ─── 2) Imports & Config ───────────────────────────────────────────────────
import os
import wikipedia
import requests
from bs4 import BeautifulSoup

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Set your OpenAI key in Colab via:
#   %env OPENAI_API_KEY=sk-...
OPENAI_API_KEY = "PASTE YOUR OPEN AI API KEY"
assert OPENAI_API_KEY, "⚠️ Please set the OPENAI_API_KEY environment variable"

# ─── 3) FAQ Data & FAISS Setup ─────────────────────────────────────────────
faqs = [
    {"question": "How do I calm anxiety attacks?",
     "answer": "Try grounding techniques: the “5-4-3-2-1” exercise, deep diaphragmatic breathing, "
               "and remind yourself that panic will pass. If it persists, reach out for support."},
    {"question": "What are nighttime anxiety coping strategies?",
     "answer": "Use progressive muscle relaxation, limit caffeine in the afternoon, maintain a bedtime ritual, "
               "and journal your worries before sleep."},
    {"question": "What is a panic attack?",
     "answer": "A panic attack is a sudden episode of intense fear with physical symptoms like heart palpitations, "
               "shortness of breath, and dizziness."},
]

# Pass the OPENAI_API_KEY directly to OpenAIEmbeddings
embeddings   = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY) # This line is changed
questions    = [f["question"] for f in faqs]
vector_store = FAISS.from_texts(questions, embeddings, metadatas=faqs)

# ─── 4) Initialize LLM ─────────────────────────────────────────────────────
# Pass the OPENAI_API_KEY to ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0.5, openai_api_key=OPENAI_API_KEY)

# ─── 5) Router Agent ───────────────────────────────────────────────────────
def route_query(query: str) -> str:
    """Routes the query to either FAQ or search based on presence in FAQs."""
    # Check if the query matches any FAQ question:
    for faq in faqs:
        if faq["question"].lower() == query.lower():
            return "faq"  # Route to FAQ if query matches

    # If not found in FAQs, route to search:
    return "search"

# ─── 6) FAQ Agent ─────────────────────────────────────────────────────────
def faq_lookup(query: str, threshold: float = 0.0):
    print("Invoking FAQ Agent")
    print("------------------")
    results = vector_store.similarity_search_with_score(query, k=3)
    top_doc, score = results[0]
    if score >= threshold:
        return top_doc.metadata["answer"]
    return None

# ─── 7) Search Agent: Wikipedia Fallback ──────────────────────────────────
def internet_search_wiki(query: str, num_pages: int = 3):
    print("Invoking Internet Search Agent")
    print("------------------")
    wikipedia.set_lang("en")
    titles = wikipedia.search(query, results=num_pages)
    snippets = []
    for title in titles:
        try:
            page = wikipedia.page(title, auto_suggest=False)
            # take the first paragraph
            first_para = page.content.split("\n\n")[0]
            snippets.append((first_para, page.url))
        except Exception:
            continue
        if len(snippets) >= num_pages:
            break

    # If no wiki pages found, fallback to a simple Google WebView scrape:
    if not snippets:
        return ("Sorry, I couldn't find relevant Wikipedia articles.", "")

    # Summarize + cite
    summary_prompt = [
        SystemMessage(content=(
            "You are an expert mental health summarizer. Given text snippets and URLs, "
            "produce a concise, empathetic answer with in-text citations [1], [2], …"
        )),
        HumanMessage(content=str(snippets))
    ]
    return llm(summary_prompt).content

# ─── 8) Orchestrator ──────────────────────────────────────────────────────
def answer_query(query: str):
    route = route_query(query)  # Store the route result
    if route == "faq":
        ans = faq_lookup(query)
        if ans:
            return f"**FAQ Answer:**\n\n{ans}"
        else:
            return "**FAQ Agent:**\n\nNo relevant FAQ found."  # Return a message indicating FAQ failure
    elif route == "search":  # Check if the route is "search"
        return internet_search_wiki(query)
    else:
        return "**Routing Error:**\n\nUnable to determine the appropriate route."  # Handle unexpected route results

# ─── 9) Example Usage ────────────────────────────────────────────────────
if __name__ == "__main__":
    print("Query 1 : How do I calm anxiety attacks?")
    print(answer_query("How do I calm anxiety attacks?"))
    print("---------------------------------------------")
    print("Query 2 : How do I treat depression?")
    print(answer_query("How do I treat depression?"))

Query 1 : How do I calm anxiety attacks?
Invoking FAQ Agent
------------------
**FAQ Answer:**

Try grounding techniques: the “5-4-3-2-1” exercise, deep diaphragmatic breathing, and remind yourself that panic will pass. If it persists, reach out for support.
---------------------------------------------
Query 2 : How do I treat depression?
Invoking Internet Search Agent
------------------
Major depressive disorder (MDD), also known as clinical depression, is a significant mental health condition characterized by persistent low mood, low self-esteem, and a loss of interest or pleasure in activities that are typically enjoyable [1]. It affects a substantial portion of the population, with onset most common in a person's 20s and affecting females more frequently than males [1]. The disorder is diagnosed through reported experiences and behaviors, as there are no laboratory tests for MDD, although tests may be conducted to exclude other conditions [1]. 

Treatment often involves psychother